In [206]:
from pprint import pprint
from PIL import Image
import piexif
import glob
from GPSPhoto import gpsphoto
import pandas as pd
import streamlit as st
import numpy as np


path = '/Users/clairemazzucato/Desktop/test2'
all_files = glob.glob(path + "/*.JPG") + glob.glob(path + "/*.jpg")
list_gps = []


def main():
    n = 0
    for filename in all_files: 
        im = Image.open(filename)
        # Get the data from image file and return a dictionary
        data = gpsphoto.getGPSData(filename)
        if data.get('Latitude') is not None and data.get('Longitude'):
            data['filename'] = filename
            list_gps.append(data)
        n = n + 1
if __name__ == '__main__':
    main()

GPSdf = pd.DataFrame(list_gps)
#display(GPSdf)

GPSdf2 = GPSdf.copy()
GPSdf2['Latitude'] = round(GPSdf2['Latitude'], 2)
GPSdf2['Longitude'] = round(GPSdf2['Longitude'], 2)
display(GPSdf2)

2022-01-12 01:18:05.817 WARNING exifread: Possibly corrupted field Tag 0x001A in MakerNote IFD


,Latitude,Longitude,Altitude,UTC-Time,Date,filename
0,-37.81,144.95,23.230814,9:9:30.65,04/05/2019,/Users/clairemazzucato/Desktop/test2/IMG_2818.JPG
1,-37.83,144.95,13.507004,9:41:12.27,07/12/2019,/Users/clairemazzucato/Desktop/test2/IMG_7960.JPG
2,-28.64,153.61,6.112938,6:56:50,04/25/2019,/Users/clairemazzucato/Desktop/test2/IMG_3284.JPG
3,-37.80,144.98,40.296605,15:49:21,04/06/2019,/Users/clairemazzucato/Desktop/test2/IMG_2830.JPG
4,-20.27,148.71,29.758389,0:51:37.99,06/01/2019,/Users/clairemazzucato/Desktop/test2/IMG_4299.JPG
...,...,...,...,...,...,...
2038,44.00,1.40,147.580350,9:59:44.13,08/23/2019,/Users/clairemazzucato/Desktop/test2/IMG_7564 ...
2039,-20.37,148.55,610.282837,4:14:52.81,06/03/2019,/Users/clairemazzucato/Desktop/test2/IMG_5198.jpg
2040,-20.42,148.70,804.636597,4:15:6.67,06/03/2019,/Users/clairemazzucato/Desktop/test2/IMG_5200.jpg
2041,-20.49,148.55,20.288864,4:8:34.08,06/03/2019,/Users/clairemazzucato/Desktop/test2/IMG_5174.jpg


In [207]:
GPSdf3 = GPSdf2.groupby(['Latitude','Longitude']).count().reset_index()
display(GPSdf3)

,Latitude,Longitude,Altitude,UTC-Time,Date,filename
0,-38.76,143.67,2,2,2,2
1,-38.67,143.10,2,2,2,2
2,-38.67,143.11,3,3,3,3
3,-38.67,143.13,1,1,1,1
4,-38.67,143.57,12,12,12,12
...,...,...,...,...,...,...
332,50.62,5.57,1,1,1,1
333,50.84,4.35,14,14,14,14
334,50.85,4.35,31,31,31,31
335,50.85,4.36,2,2,2,2


In [208]:
# importing necessary libraries 
import reverse_geocoder as rg 

def reverseGeocode(coordinates): 
    result = rg.search(coordinates) 
    
    return result

# the code belows is used to find the location infos according to GPS coordinates 
if __name__ == "__main__":
    result = []
    for i in range(0, len(GPSdf3)):
        coordinates =(GPSdf3["Latitude"][i], GPSdf3["Longitude"][i]) 
        result.append(reverseGeocode(coordinates))
    #print(result)
    
    for i in range(0, len(result)):
        test1=pd.DataFrame(result)
#Renamed the header of the single column into 'Location'
        test1.columns =['Location']

#the code belows split the single column of test1 dataframe into multiple columns. 
test2 = test1['Location'].apply(pd.Series)

#Let's concatenate the GPSdf (gps coordinates df) and test2 (dataframe of location information)
newdf = pd.concat ([GPSdf3, test2], axis=1)

#The code belows remove the unecessary columns 
#newdf = newdf.drop(['lat', 'lon','name','admin1'], axis=1)

In [209]:
#Let's concatenate the GPSdf (gps coordinates df) and test2 (dataframe of location information)
newdf = pd.concat ([GPSdf3, test2], axis=1)

In [210]:
newdf = newdf.replace(to_replace=r'AU', value='Australia', regex=True)
newdf = newdf.replace(to_replace=r'PH', value='Philippines', regex=True)
newdf = newdf.replace(to_replace=r'FR', value='France', regex=True)
newdf = newdf.replace(to_replace=r'BE', value='Belgium', regex=True)
newdf = newdf.replace(to_replace=r'ID', value='Indonesia', regex=True)
newdf = newdf.replace(to_replace=r'CN', value='China', regex=True)
newdf = newdf.replace(to_replace=r'NL', value='Netherlands', regex=True)
newdf = newdf.replace(to_replace=r'SE', value='Sweden', regex=True)
newdf = newdf.replace(to_replace=r'US', value='United States', regex=True)
newdf = newdf.replace(to_replace=r'ES', value='Spain', regex=True)


newdf = newdf.rename(columns={'admin2': 'City', 'cc': 'Country'})
display(newdf)

,Latitude,Longitude,Altitude,UTC-Time,Date,filename,lat,lon,name,admin1,City,Country
0,-38.76,143.67,2,2,2,2,-38.7594,143.67219,Apollo Bay,Victoria,Colac-Otway,Australia
1,-38.67,143.10,2,2,2,2,-38.7594,143.67219,Apollo Bay,Victoria,Colac-Otway,Australia
2,-38.67,143.11,3,3,3,3,-38.7594,143.67219,Apollo Bay,Victoria,Colac-Otway,Australia
3,-38.67,143.13,1,1,1,1,-38.7594,143.67219,Apollo Bay,Victoria,Colac-Otway,Australia
4,-38.67,143.57,12,12,12,12,-38.7594,143.67219,Apollo Bay,Victoria,Colac-Otway,Australia
...,...,...,...,...,...,...,...,...,...,...,...,...
332,50.62,5.57,1,1,1,1,50.63373,5.56749,Liege,Wallonia,Province de Liege,Belgium
333,50.84,4.35,14,14,14,14,50.85045,4.34878,Brussels,Brussels Capital,Bruxelles-Capitale,Belgium
334,50.85,4.35,31,31,31,31,50.85045,4.34878,Brussels,Brussels Capital,Bruxelles-Capitale,Belgium
335,50.85,4.36,2,2,2,2,50.85045,4.34878,Brussels,Brussels Capital,Bruxelles-Capitale,Belgium


In [226]:
newdf.to_csv('2019photos.csv') 

In [227]:
GPSdf2.to_csv('2019photos-bis.csv')

In [211]:
newdf_sum = newdf.groupby(['City']).sum()
newdf_sort = newdf_sum.sort_values('filename', ascending=False).head(10)
newdf_sort = newdf_sort.iloc[1: , :]
display(newdf_sort)

,Latitude,Longitude,Altitude,UTC-Time,Date,filename
City,,,,,,
Melbourne,-415.90,1594.44,314,314,314,314
Whitsunday,-1515.02,11166.28,278,278,278,278
Yorke Peninsula,-734.83,2881.79,100,100,100,100
Mornington Peninsula,-497.89,1883.34,76,76,76,76
City of Sydney,-203.23,907.25,70,70,70,70
Port Phillip,-378.49,1449.71,66,66,66,66
Fraser Coast,-356.03,2143.68,61,61,61,61
Sunshine Coast,-238.24,1377.81,57,57,57,57
Waverley,-169.49,756.37,53,53,53,53


In [212]:
import datetime as dt
#from dateutil.relativedelta import relativedelta # to add days or years

GPSdf2['Date'] = pd.to_datetime(GPSdf2['Date'])
GPSdf_month = GPSdf2.set_index('Date')
GPSdf2_month = GPSdf_month.groupby(pd.Grouper(freq="M")).count().reset_index()
start_date = dt.datetime.now().date()-relativedelta(years=3)#  I need some range in the past
end_date = dt.datetime.now().date()-relativedelta(years=2)
GPSdf2_month = GPSdf2_month.loc[GPSdf2_month['Date'].dt.date > start_date]
GPSdf2_month = GPSdf2_month.loc[GPSdf2_month['Date'].dt.date < end_date]
display(GPSdf2_month)

,Date,Latitude,Longitude,Altitude,UTC-Time,filename
588,2019-01-31,0,0,0,0,0
589,2019-02-28,351,351,350,351,351
590,2019-03-31,225,225,225,225,225
591,2019-04-30,269,269,269,269,269
592,2019-05-31,109,109,108,109,109
593,2019-06-30,763,763,763,763,763
594,2019-07-31,179,179,178,179,179
595,2019-08-31,40,40,40,40,40
596,2019-09-30,70,70,70,70,70
597,2019-10-31,0,0,0,0,0


In [213]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [214]:
import plotly.express as px
fig = px.bar(GPSdf2_month, x='Date', y='filename')
fig.show()

In [215]:
def adding_dt_info(df):
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['day'] = df['Date'].dt.day
    df['Hour'] = df['Date'].dt.hour
    df['Weekday'] = df['Date'].dt.strftime("%A")
    df['nb_week'] = df['Date'].dt.isocalendar().week
    return df

In [216]:
GPSdf2['Date'] = pd.to_datetime(GPSdf2['Date'])
GPSdf2 = adding_dt_info(GPSdf2)
GPSdf_month = GPSdf2.set_index('Date')
GPSdf2_month = GPSdf_month.groupby(pd.Grouper(freq="W")).count().reset_index()
start_date = dt.datetime.now().date()-relativedelta(years=3)#  I need some range in the past
end_date = dt.datetime.now().date()-relativedelta(years=2)
GPSdf2_month = GPSdf2_month.loc[GPSdf2_month['Date'].dt.date > start_date]
GPSdf2_month = GPSdf2_month.loc[GPSdf2_month['Date'].dt.date < end_date]
display(GPSdf2_month)

,Date,Latitude,Longitude,Altitude,UTC-Time,filename,Year,Month,day,Hour,Weekday,nb_week
2558,2019-01-13,0,0,0,0,0,0,0,0,0,0,0
2559,2019-01-20,0,0,0,0,0,0,0,0,0,0,0
2560,2019-01-27,0,0,0,0,0,0,0,0,0,0,0
2561,2019-02-03,5,5,5,5,5,5,5,5,5,5,5
2562,2019-02-10,74,74,73,74,74,74,74,74,74,74,74
2563,2019-02-17,43,43,43,43,43,43,43,43,43,43,43
2564,2019-02-24,93,93,93,93,93,93,93,93,93,93,93
2565,2019-03-03,156,156,156,156,156,156,156,156,156,156,156
2566,2019-03-10,22,22,22,22,22,22,22,22,22,22,22
2567,2019-03-17,95,95,95,95,95,95,95,95,95,95,95


In [224]:
fig = px.density_heatmap(GPSdf2, x="Month", y="Weekday")
fig.show()

In [225]:
Totalphotos = GPSdf2['filename'].count()
Totalphotos

2043

In [174]:
Totalphotos = Totalphotos.astype(int)

In [175]:
Totalphotos

1868

In [176]:
type(Totalphotos)

numpy.int64

In [194]:
List_countries = newdf.groupby(['Country']).count()
display(List_countries)
List_countries1 = List_countries[['City']]
List_countries1

,Latitude,Longitude,Altitude,UTC-Time,Date,filename,lat,lon,name,admin1,City,Indexes
Country,,,,,,,,,,,,
Australia,165,165,165,165,165,165,165,165,165,165,165,165
Belgium,5,5,5,5,5,5,5,5,5,5,5,5
China,6,6,6,6,6,6,6,6,6,6,6,6
France,33,33,33,33,33,33,33,33,33,33,33,33
Indonesia,43,43,43,43,43,43,43,43,43,43,43,43
Netherlands,1,1,1,1,1,1,1,1,1,1,1,1
Philippines,31,31,31,31,31,31,31,31,31,31,31,31
Spain,1,1,1,1,1,1,1,1,1,1,1,1
Sweden,1,1,1,1,1,1,1,1,1,1,1,1


,City
Country,
Australia,165
Belgium,5
China,6
France,33
Indonesia,43
Netherlands,1
Philippines,31
Spain,1
Sweden,1
